# Домашняя работа

**Задание к dataset "Такси"**

Проверить, как разные типы преобразования категориальных переменных влияют на метрику **roc_auc_score.**
Название функций:
1. **get_one_hot**
2. **get_target_share**
3. **get_woe_v1**


**Задание к dataset "Медиа"**
1. Разделить dataset на train и test в соотношении 50:50 70:30 80:20 (с перемешиванием)
2. Обучать наши модели на train. Предсказывать и замерять метрику R2 на train и на test.
3. Проверить следующие модели для каждого разделения:
a) Sales - log_tv + radio
b) Sales - TV + radio
c) Sales - radio + newspaper

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, r2_score, roc_auc_score, f1_score, precision_score, recall_score
import math

**Загрузка исходных данных**

In [2]:
taxi = pd.read_csv('taxi.csv')
taxi.head()

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,-1.000,-1.000,Economy,private,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,Standard,private,1
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,Economy,private,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,-1.000,-1.000,Economy,private,1
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,Economy,private,1


In [3]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
offer_gk                  100000 non-null int64
weekday_key               100000 non-null int64
hour_key                  100000 non-null int64
driver_gk                 100000 non-null int64
order_gk                  100000 non-null int64
driver_latitude           100000 non-null float64
driver_longitude          100000 non-null float64
origin_order_latitude     100000 non-null float64
origin_order_longitude    100000 non-null float64
distance_km               100000 non-null float64
duration_min              100000 non-null float64
offer_class_group         100000 non-null object
ride_type_desc            100000 non-null object
driver_response           100000 non-null int64
dtypes: float64(6), int64(6), object(2)
memory usage: 9.9+ MB


#### 1. Преобразование категориальной переменной с помощью get one hot

In [4]:
# Преобразование категориальной переменной 'offer class group' с помощью метода 'get_one_hot'. 
# (Переменная 'ride_type_desc' была убрана с целью приведения дата-сета к единообразию в дальнейшей работе \ 
# с другими методами преобразования переменных get_target_share и get_woe_v1, с целью последующего сравнения метрики roc_auc)
df_one_hot = taxi.drop('ride_type_desc', axis=1)
one_hot_df = pd.get_dummies(df_one_hot)
one_hot_df.head()

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,...,driver_response,offer_class_group_Delivery,offer_class_group_Economy,offer_class_group_Kids,offer_class_group_Premium,offer_class_group_Standard,offer_class_group_Test,offer_class_group_VIP,offer_class_group_VIP+,offer_class_group_XL
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,-1.000,...,0,0,1,0,0,0,0,0,0,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,...,1,0,0,0,0,1,0,0,0,0
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,...,0,0,1,0,0,0,0,0,0,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,-1.000,...,1,0,1,0,0,0,0,0,0,0
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,...,1,0,1,0,0,0,0,0,0,0


In [5]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки.
y=one_hot_df['driver_response'].copy()
X=one_hot_df.drop('driver_response', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# Стандартизация значений переменных
scaler = StandardScaler()
scaler.fit_transform(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with

In [7]:
# Модель RandomForestClassifier (случайный лес)
RFC = RandomForestClassifier() # выбираем модель обучения (логистическая регрессия)
RFC.fit(X_train, y_train) # обучаем модель

y_pred_RFC_test = RFC.predict_proba(X_test)[:,1] # предсказание для train
y_pred_RFC_train = RFC.predict_proba(X_train)[:,1] # предсказание для test

roc_auc_test = np.round(roc_auc_score(y_test, y_pred_RFC_test), 2)
roc_auc_train = np.round(roc_auc_score(y_train, y_pred_RFC_train), 2)

print("one_hot_roc_auc_Train: ", roc_auc_train)
print("one_hot_roc_auc_Test: ", roc_auc_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


one_hot_roc_auc_Train:  1.0
one_hot_roc_auc_Test:  0.85


#### 2. Преобразование категориальной переменной с помощью get_target_share

In [8]:
# Преобразование категориальной переменной 'offer class group' с помощью метода 'get_target_share'
offer_class_group_dict = (
    taxi[taxi['driver_response'] == 1]['offer_class_group'].value_counts()
    /
    taxi['offer_class_group'].value_counts()
).to_dict()

In [9]:
# Добавление новой преобразованной переменной в дата-сет
taxi['offer_class_group'].apply(offer_class_group_dict.get).head()

0    0.439040
1    0.523494
2    0.439040
3    0.439040
4    0.439040
Name: offer_class_group, dtype: float64

In [10]:
# Приведение дата-сета к единообразию, для сравнения метрики roc_auc
taxi1 = taxi.drop('offer_class_group', axis=1)
df_target = taxi1.drop('ride_type_desc', axis=1)

In [11]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки.
y1=df_target['driver_response'].copy()
X1=df_target.drop('driver_response', axis=1)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [12]:
# Стандартизация значений переменных
scaler = StandardScaler()
scaler.fit_transform(X1_train)
X1_train_scaled = scaler.transform(X1_train)
X1_test_scaled = scaler.fit_transform(X1_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 

In [13]:
# Модель RandomForestClassifier (случайный лес)
RFC_target = RandomForestClassifier() # выбираем модель обучения (логистическая регрессия)
RFC_target.fit(X1_train, y1_train) # обучаем модель

y1_pred_RFC_test = RFC_target.predict_proba(X1_test)[:,1] # предсказание для train
y1_pred_RFC_train = RFC_target.predict_proba(X1_train)[:,1] # предсказание для test

target_roc_auc_test = np.round(roc_auc_score(y1_test, y1_pred_RFC_test), 2)
target_roc_auc_train = np.round(roc_auc_score(y1_train, y1_pred_RFC_train), 2)

print("RFC_target_roc_auc_Train: ", target_roc_auc_train)
print("RFC_target_roc_auc_Test: ", target_roc_auc_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RFC_target_roc_auc_Train:  1.0
RFC_target_roc_auc_Test:  0.85


#### 3. Преобразование категориальной переменной с помощью get_woe_v1

In [14]:
# Преобразование категориальной переменной 'offer class group' с помощью метода 'get_woe_v1'
all_good = len(taxi[taxi['driver_response'] == 1]['offer_class_group'])
all_bad = len(taxi[taxi['driver_response'] == 0]['offer_class_group'])
odds_series = (taxi[taxi['driver_response'] == 1]['offer_class_group'].value_counts() / taxi[taxi['driver_response'] == 0]['offer_class_group'].value_counts())
odd_series = odds_series = odds_series / all_good * all_bad
category_woe_dict = np.log(odds_series).to_dict()

In [15]:
# Добавление новой преобразованной переменной в дата-сет
taxi['offer_class_group'] = taxi['offer_class_group'].apply(category_woe_dict.get)

In [16]:
# Приведение дата-сета к единообразию, для сравнения метрики roc_auc
taxi2 = taxi.drop('offer_class_group', axis=1)
df_woe = taxi2.drop('ride_type_desc', axis=1)

In [17]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки.
y2=df_woe['driver_response'].copy()
X2=df_woe.drop('driver_response', axis=1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

In [18]:
# Стандартизация значений переменных
scaler = StandardScaler()
scaler.fit_transform(X2_train)
X2_train_scaled = scaler.transform(X2_train)
X2_test_scaled = scaler.fit_transform(X2_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 

In [19]:
# Модель RandomForestClassifier (случайный лес)
RFC_woe = RandomForestClassifier() # выбираем модель обучения (логистическая регрессия)
RFC_woe.fit(X1_train, y1_train) # обучаем модель

y2_pred_RFC_test = RFC_woe.predict_proba(X1_test)[:,1] # предсказание для train
y2_pred_RFC_train = RFC_woe.predict_proba(X1_train)[:,1] # предсказание для test

woe_roc_auc_test = np.round(roc_auc_score(y2_test, y2_pred_RFC_test), 2)
woe_roc_auc_train = np.round(roc_auc_score(y2_train, y2_pred_RFC_train), 2)

print("RFC_woe_roc_auc_Train: ", target_roc_auc_train)
print("RFC_woe_roc_auc_Test: ", target_roc_auc_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RFC_woe_roc_auc_Train:  1.0
RFC_woe_roc_auc_Test:  0.85


In [20]:
print("one_hot_roc_auc_Train: ", roc_auc_train)
print("target_share_roc_auc_Train: ", target_roc_auc_train)
print("woe_roc_auc_Train: ", target_roc_auc_train)
print("")
print("one_hot_roc_auc_Test: ", roc_auc_test)
print("target_share_roc_auc_Test: ", target_roc_auc_test)
print("woe_roc_auc_Test: ", target_roc_auc_test)

one_hot_roc_auc_Train:  1.0
target_share_roc_auc_Train:  1.0
woe_roc_auc_Train:  1.0

one_hot_roc_auc_Test:  0.85
target_share_roc_auc_Test:  0.85
woe_roc_auc_Test:  0.85


**Изменение методов преобразования категориальных переменных не повлияло на изменение метрики roc_auc.**

### Задание к dataset "Медиа"

**Загрузка исходных данных**

In [21]:
media = pd.read_csv('Advertising.csv')
media = media.drop('Unnamed: 0', axis = 1)
media.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [22]:
media.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
TV           200 non-null float64
radio        200 non-null float64
newspaper    200 non-null float64
sales        200 non-null float64
dtypes: float64(4)
memory usage: 6.3 KB


In [23]:
media['log_tv'] = media.TV.apply(lambda x: math.log(x, 2))

**Разделить dataset на train и test в соотношении 50:50 70:30 80:20 (с перемешиванием)**

In [24]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки (50:50).
X_train, X_test, y_train, y_test = train_test_split(\
media[['TV', 'radio', 'newspaper', 'log_tv']], media['sales'], test_size=0.5)

In [25]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки (70:30).
X1_train, X1_test, y1_train, y1_test = train_test_split(\
media[['TV', 'radio', 'newspaper', 'log_tv']], media['sales'], test_size=0.3)

In [26]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки (80:20).
X2_train, X2_test, y2_train, y2_test = train_test_split(\
media[['TV', 'radio', 'newspaper', 'log_tv']], media['sales'], test_size=0.2)

**Обучать наши модели на train. Предсказывать и замерять метрику R2 на train и на test.**

In [27]:
# Обучение модели на train. Предсказание и замер метрики R^2 на train и на test (Выборка 50:50)
# Обучение модели на train.
reg_50_50 = LinearRegression().fit(X_train, y_train)

# Предсказание target на train и на test
y_predict_test_50_50 = reg_50_50.predict(X_test)
y_predict_train_50_50 = reg_50_50.predict(X_train)

# Замер метрики R^2 на train и на test
r2_score_test_50_50 = r2_score(y_test, y_predict_test_50_50.round())
r2_score_train_50_50 = r2_score(y_train, y_predict_train_50_50.round())
print(f'R^2_test (50_50): {r2_score_test_50_50:.2f}')
print(f'R^2_train (50_50): {r2_score_train_50_50:.2f}')

R^2_test (50_50): 0.93
R^2_train (50_50): 0.93


In [28]:
# Обучение модели на train. Предсказание и замер метрики R^2 на train и на test (Выборка 70:30)
# Обучение модели на train.
reg_70_30 = LinearRegression().fit(X1_train, y1_train)

# Предсказание target на train и на test
y_predict_test_70_30 = reg_70_30.predict(X1_test)
y_predict_train_70_30 = reg_70_30.predict(X1_train)

# Замер метрики R^2 на train и на test
r2_score_test_70_30 = r2_score(y1_test, y_predict_test_70_30.round())
r2_score_train_70_30 = r2_score(y1_train, y_predict_train_70_30.round())
print(f'R^2_test (70_30): {r2_score_test_70_30:.2f}')
print(f'R^2_train (70_30): {r2_score_train_70_30:.2f}')

R^2_test (70_30): 0.91
R^2_train (70_30): 0.94


In [29]:
# Обучение модели на train. Предсказание и замер метрики R^2 на train и на test (Выборка 80:20)
# Обучение модели на train.
reg_80_20 = LinearRegression().fit(X2_train, y2_train)

# Предсказание target на train и на test
y_predict_test_80_20 = reg_80_20.predict(X2_test)
y_predict_train_80_20 = reg_80_20.predict(X2_train)

# Замер метрики R^2 на train и на test
r2_score_test_80_20 = r2_score(y2_test, y_predict_test_80_20.round())
r2_score_train_80_20 = r2_score(y2_train, y_predict_train_80_20.round())
print(f'R^2_test (80_20): {r2_score_test_80_20:.2f}')
print(f'R^2_train (80_20): {r2_score_train_80_20:.2f}')

R^2_test (80_20): 0.94
R^2_train (80_20): 0.93


**Проверить следующие модели для каждого разделения:**
1. sales ~ log_tv + radio
2. sales ~ TV + radio
3. sales ~ radio + newspaper

**1. Разделение:** ***sales ~ log_tv + radio***

In [30]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки
# (50:50).
X_train, X_test, y_train, y_test = train_test_split(\
media[['radio', 'log_tv']], media['sales'], test_size=0.5, random_state=42)

# (70:30).
X1_train, X1_test, y1_train, y1_test = train_test_split(\
media[['radio', 'log_tv']], media['sales'], test_size=0.3, random_state=42)

# (80:20).
X2_train, X2_test, y2_train, y2_test = train_test_split(\
media[['radio', 'log_tv']], media['sales'], test_size=0.2, random_state=42)

In [31]:
# Обучение модели на train. Предсказание и замер метрики R^2 на train и на test 

#(Выборка 50:50)
# Обучение модели на train.
reg_50_50 = LinearRegression().fit(X_train, y_train)
# Предсказание target на train и на test
y_predict_test_50_50 = reg_50_50.predict(X_test)
y_predict_train_50_50 = reg_50_50.predict(X_train)
# Замер метрики R^2 на train и на test
r2_score_test_50_50 = r2_score(y_test, y_predict_test_50_50.round())
r2_score_train_50_50 = r2_score(y_train, y_predict_train_50_50.round())

# (Выборка 70:30)
# Обучение модели на train.
reg_70_30 = LinearRegression().fit(X1_train, y1_train)
# Предсказание target на train и на test
y_predict_test_70_30 = reg_70_30.predict(X1_test)
y_predict_train_70_30 = reg_70_30.predict(X1_train)
# Замер метрики R^2 на train и на test
r2_score_test_70_30 = r2_score(y1_test, y_predict_test_70_30.round())
r2_score_train_70_30 = r2_score(y1_train, y_predict_train_70_30.round())


# (Выборка 80:20)
# Обучение модели на train.
reg_80_20 = LinearRegression().fit(X2_train, y2_train)
# Предсказание target на train и на test
y_predict_test_80_20 = reg_80_20.predict(X2_test)
y_predict_train_80_20 = reg_80_20.predict(X2_train)
# Замер метрики R^2 на train и на test
r2_score_test_80_20 = r2_score(y2_test, y_predict_test_80_20.round())
r2_score_train_80_20 = r2_score(y2_train, y_predict_train_80_20.round())

# Вывод результатов по метрике R^2
print(f'R^2_test (50_50): {r2_score_test_50_50:.2f}')
print(f'R^2_train (50_50): {r2_score_train_50_50:.2f}')
print('')
print(f'R^2_test (70_30): {r2_score_test_70_30:.2f}')
print(f'R^2_train (70_30): {r2_score_train_70_30:.2f}')
print('')
print(f'R^2_test (80_20): {r2_score_test_80_20:.2f}')
print(f'R^2_train (80_20): {r2_score_train_80_20:.2f}')

R^2_test (50_50): 0.90
R^2_train (50_50): 0.91

R^2_test (70_30): 0.91
R^2_train (70_30): 0.90

R^2_test (80_20): 0.92
R^2_train (80_20): 0.90


**2. Разделение:** ***sales ~ TV + radio***

In [32]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки
# (50:50).
X_train, X_test, y_train, y_test = train_test_split(\
media[['TV', 'radio']], media['sales'], test_size=0.5, random_state=42)

# (70:30).
X1_train, X1_test, y1_train, y1_test = train_test_split(\
media[['TV', 'radio']], media['sales'], test_size=0.3, random_state=42)

# (80:20).
X2_train, X2_test, y2_train, y2_test = train_test_split(\
media[['TV', 'radio']], media['sales'], test_size=0.2, random_state=42)

In [33]:
# Обучение модели на train. Предсказание и замер метрики R^2 на train и на test 

#(Выборка 50:50)
# Обучение модели на train.
reg_50_50 = LinearRegression().fit(X_train, y_train)
# Предсказание target на train и на test
y_predict_test_50_50 = reg_50_50.predict(X_test)
y_predict_train_50_50 = reg_50_50.predict(X_train)
# Замер метрики R^2 на train и на test
r2_score_test_50_50 = r2_score(y_test, y_predict_test_50_50.round())
r2_score_train_50_50 = r2_score(y_train, y_predict_train_50_50.round())

# (Выборка 70:30)
# Обучение модели на train.
reg_70_30 = LinearRegression().fit(X1_train, y1_train)
# Предсказание target на train и на test
y_predict_test_70_30 = reg_70_30.predict(X1_test)
y_predict_train_70_30 = reg_70_30.predict(X1_train)
# Замер метрики R^2 на train и на test
r2_score_test_70_30 = r2_score(y1_test, y_predict_test_70_30.round())
r2_score_train_70_30 = r2_score(y1_train, y_predict_train_70_30.round())


# (Выборка 80:20)
# Обучение модели на train.
reg_80_20 = LinearRegression().fit(X2_train, y2_train)
# Предсказание target на train и на test
y_predict_test_80_20 = reg_80_20.predict(X2_test)
y_predict_train_80_20 = reg_80_20.predict(X2_train)
# Замер метрики R^2 на train и на test
r2_score_test_80_20 = r2_score(y2_test, y_predict_test_80_20.round())
r2_score_train_80_20 = r2_score(y2_train, y_predict_train_80_20.round())

# Вывод результатов по метрике R^2
print(f'R^2_test (50_50): {r2_score_test_50_50:.2f}')
print(f'R^2_train (50_50): {r2_score_train_50_50:.2f}')
print('')
print(f'R^2_test (70_30): {r2_score_test_70_30:.2f}')
print(f'R^2_train (70_30): {r2_score_train_70_30:.2f}')
print('')
print(f'R^2_test (80_20): {r2_score_test_80_20:.2f}')
print(f'R^2_train (80_20): {r2_score_train_80_20:.2f}')

R^2_test (50_50): 0.88
R^2_train (50_50): 0.90

R^2_test (70_30): 0.86
R^2_train (70_30): 0.90

R^2_test (80_20): 0.90
R^2_train (80_20): 0.89


**3. Разделение:** ***sales ~ radio + newspaper***

In [34]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки
# (50:50).
X_train, X_test, y_train, y_test = train_test_split(\
media[['radio', 'newspaper']], media['sales'], test_size=0.5, random_state=42)

# (70:30).
X1_train, X1_test, y1_train, y1_test = train_test_split(\
media[['radio', 'newspaper']], media['sales'], test_size=0.3, random_state=42)

# (80:20).
X2_train, X2_test, y2_train, y2_test = train_test_split(\
media[['radio', 'newspaper']], media['sales'], test_size=0.2, random_state=42)

In [35]:
# Обучение модели на train. Предсказание и замер метрики R^2 на train и на test 

#(Выборка 50:50)
# Обучение модели на train.
reg_50_50 = LinearRegression().fit(X_train, y_train)
# Предсказание target на train и на test
y_predict_test_50_50 = reg_50_50.predict(X_test)
y_predict_train_50_50 = reg_50_50.predict(X_train)
# Замер метрики R^2 на train и на test
r2_score_test_50_50 = r2_score(y_test, y_predict_test_50_50.round())
r2_score_train_50_50 = r2_score(y_train, y_predict_train_50_50.round())

# (Выборка 70:30)
# Обучение модели на train.
reg_70_30 = LinearRegression().fit(X1_train, y1_train)
# Предсказание target на train и на test
y_predict_test_70_30 = reg_70_30.predict(X1_test)
y_predict_train_70_30 = reg_70_30.predict(X1_train)
# Замер метрики R^2 на train и на test
r2_score_test_70_30 = r2_score(y1_test, y_predict_test_70_30.round())
r2_score_train_70_30 = r2_score(y1_train, y_predict_train_70_30.round())


# (Выборка 80:20)
# Обучение модели на train.
reg_80_20 = LinearRegression().fit(X2_train, y2_train)
# Предсказание target на train и на test
y_predict_test_80_20 = reg_80_20.predict(X2_test)
y_predict_train_80_20 = reg_80_20.predict(X2_train)
# Замер метрики R^2 на train и на test
r2_score_test_80_20 = r2_score(y2_test, y_predict_test_80_20.round())
r2_score_train_80_20 = r2_score(y2_train, y_predict_train_80_20.round())

# Вывод результатов по метрике R^2
print(f'R^2_test (50_50): {r2_score_test_50_50:.2f}')
print(f'R^2_train (50_50): {r2_score_train_50_50:.2f}')
print('')
print(f'R^2_test (70_30): {r2_score_test_70_30:.2f}')
print(f'R^2_train (70_30): {r2_score_train_70_30:.2f}')
print('')
print(f'R^2_test (80_20): {r2_score_test_80_20:.2f}')
print(f'R^2_train (80_20): {r2_score_train_80_20:.2f}')

R^2_test (50_50): 0.14
R^2_train (50_50): 0.46

R^2_test (70_30): 0.13
R^2_train (70_30): 0.39

R^2_test (80_20): 0.25
R^2_train (80_20): 0.34


**Ссылка на репозиторий:** https://github.com/OleginV/DVP-3

**Название блокнота:** ДЗ-2 Feature Selection.ipynb